MODEL USED : KNN
DATASET : ALL WEEKS (input_2023_w01 to w18 / output_2023_w01 to w18)
OUTPUT FRAMES PREDICTED : 15 
INPUT FRAMES NEEDED BEFORE THROW : 8 (X-7, X-6,..., X0) 
METHOD : MULTIVARIATE 
SEPARATE MODELS FOR X AND Y : NO


In [26]:
import os
import re
import glob
import json
import joblib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import lightgbm as lgb
from sklearn.neighbors import KDTree
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Load ALL input datasets (w01 to w18)
print("Loading all input datasets (w01 to w18)...")
print("="*80)

input_dfs = []
for week in range(1, 19):
    filename = f'train/input_2023_w{week:02d}.csv'
    print(f"Loading {filename}...", end=" ")
    df = pd.read_csv(filename)
    print(f"✓ Shape: {df.shape}")
    input_dfs.append(df)

# Concatenate all input dataframes
df_input = pd.concat(input_dfs, ignore_index=True)
print(f"\n✓ All input datasets loaded and merged!")
print(f"  Total shape: {df_input.shape}")
print(f"  Unique games: {df_input['game_id'].nunique()}")
print(f"  Unique players: {df_input['nfl_id'].nunique()}")

# Load ALL output datasets (w01 to w18)
print("\n" + "="*80)
print("Loading all output datasets (w01 to w18)...")
print("="*80)

output_dfs = []
for week in range(1, 19):
    filename = f'train/output_2023_w{week:02d}.csv'
    print(f"Loading {filename}...", end=" ")
    df = pd.read_csv(filename)
    print(f"✓ Shape: {df.shape}")
    output_dfs.append(df)

# Concatenate all output dataframes
df_output = pd.concat(output_dfs, ignore_index=True)
print(f"\n✓ All output datasets loaded and merged!")
print(f"  Total shape: {df_output.shape}")
print(f"  Unique games: {df_output['game_id'].nunique()}")
print(f"  Unique players: {df_output['nfl_id'].nunique()}")

print("\n" + "="*80)
print("DATA LOADING COMPLETE")
print("="*80)
print(f"df_input:  {df_input.shape}")
print(f"df_output: {df_output.shape}")

# Display sample
print("\nSample of df_input:")
df_input.head()







Loading all input datasets (w01 to w18)...
Loading train/input_2023_w01.csv... ✓ Shape: (285714, 23)
Loading train/input_2023_w02.csv... ✓ Shape: (288586, 23)
Loading train/input_2023_w03.csv... ✓ Shape: (297757, 23)
Loading train/input_2023_w04.csv... ✓ Shape: (272475, 23)
Loading train/input_2023_w05.csv... ✓ Shape: (254779, 23)
Loading train/input_2023_w06.csv... ✓ Shape: (270676, 23)
Loading train/input_2023_w07.csv... ✓ Shape: (233597, 23)
Loading train/input_2023_w08.csv... ✓ Shape: (281011, 23)
Loading train/input_2023_w09.csv... ✓ Shape: (252796, 23)
Loading train/input_2023_w10.csv... ✓ Shape: (260372, 23)
Loading train/input_2023_w11.csv... ✓ Shape: (243413, 23)
Loading train/input_2023_w12.csv... ✓ Shape: (294940, 23)
Loading train/input_2023_w13.csv... ✓ Shape: (233755, 23)
Loading train/input_2023_w14.csv... ✓ Shape: (279972, 23)
Loading train/input_2023_w15.csv... ✓ Shape: (281820, 23)
Loading train/input_2023_w16.csv... ✓ Shape: (316417, 23)
Loading train/input_2023_w17.

,game_id,play_id,player_to_predict,nfl_id,frame_id,play_direction,absolute_yardline_number,player_name,player_height,player_weight,...,player_role,x,y,s,a,dir,o,num_frames_output,ball_land_x,ball_land_y
0,2023090700,101,False,54527,1,right,42,Bryan Cook,6-1,210,...,Defensive Coverage,52.33,36.94,0.09,0.39,322.40,238.24,21,63.259998,-0.22
1,2023090700,101,False,54527,2,right,42,Bryan Cook,6-1,210,...,Defensive Coverage,52.33,36.94,0.04,0.61,200.89,236.05,21,63.259998,-0.22
2,2023090700,101,False,54527,3,right,42,Bryan Cook,6-1,210,...,Defensive Coverage,52.33,36.93,0.12,0.73,147.55,240.60,21,63.259998,-0.22
3,2023090700,101,False,54527,4,right,42,Bryan Cook,6-1,210,...,Defensive Coverage,52.35,36.92,0.23,0.81,131.40,244.25,21,63.259998,-0.22
4,2023090700,101,False,54527,5,right,42,Bryan Cook,6-1,210,...,Defensive Coverage,52.37,36.90,0.35,0.82,123.26,244.25,21,63.259998,-0.22


In [32]:
df_input_TRUE = df_input[df_input['player_to_predict'] == True]
df_input_TRUE.shape

(1303440, 23)

In [34]:
subjects = df_input_TRUE[['game_id', 'play_id', 'nfl_id']].drop_duplicates()
subjects.shape


(46045, 3)

In [33]:
# Count how many output frames each player actually has
frame_counts = df_output.groupby(['game_id', 'play_id', 'nfl_id']).size()

# Distribution: number of players with exactly k frames
freq = frame_counts.value_counts().sort_index()

print("Exact frame count distribution:\n", freq)

# Compute cumulative: players with at least k frames
cumulative_at_least = freq[::-1].cumsum()[::-1]

print("\nCumulative frequency (players with at least k frames):\n", cumulative_at_least)


Exact frame count distribution:
 15    11270
Name: count, dtype: int64

Cumulative frequency (players with at least k frames):
 15    11270
Name: count, dtype: int64


In [28]:

# Check what's the lowest number of available input frames


s = df_input_TRUE['frame_id'] 

# Identify starts of new sequences: whenever the value decreases
groups = (s < s.shift()).cumsum()

# Compute the maximum in each group (i.e., the "last" value)
last_vals = s.groupby(groups).max()

# Get the smallest last value
result = last_vals.min()

print(result)


8


In [29]:
df_input_TRUE = df_input_TRUE.drop(columns=["player_name"])

In [30]:
# Step 1: Keep ALL players, but for each player, keep only their last 8 input frames
print(f"Before filtering - df_input_TRUE shape: {df_input_TRUE.shape}")
print(f"Before filtering - df_output shape: {df_output.shape}")
print(f"num_frames_output distribution:\n{df_input_TRUE['num_frames_output'].value_counts().sort_index()}")

# Get the last 8 frame_ids for each player (sorted descending, take top 8)
last_8_frames = df_input_TRUE.groupby(['game_id', 'play_id', 'nfl_id']).apply(
    lambda x: x.nlargest(8, 'frame_id')[['game_id', 'play_id', 'nfl_id', 'frame_id']]
).reset_index(drop=True)

print(f"\nLast 8 frames shape: {last_8_frames.shape}")
print(f"Frames per player:\n{last_8_frames.groupby(['game_id', 'play_id', 'nfl_id']).size().value_counts()}")

# Keep only rows that match the last 8 frames for each player
df_input_TRUE = df_input_TRUE.merge(
    last_8_frames, 
    on=['game_id', 'play_id', 'nfl_id', 'frame_id'], 
    how='inner'
)

print(f"After keeping only last 8 input frames per player: {df_input_TRUE.shape}")

# Step 2: Filter df_output to keep ONLY players with AT LEAST 15 output frames (cap at 15)
print(f"\n--- Processing df_output ---")
print(f"df_output frame_id range: {df_output['frame_id'].min()} to {df_output['frame_id'].max()}")

# Keep only output frames 1 through 15
df_output = df_output[df_output['frame_id'] <= 15].copy()

# Count how many output frames each player has
output_frame_counts = df_output.groupby(['game_id', 'play_id', 'nfl_id']).size()
print(f"\nOutput frame counts distribution:")
print(output_frame_counts.value_counts().sort_index())

# Get players with AT LEAST 15 output frames
players_with_15plus_frames = output_frame_counts[output_frame_counts >= 15].index
print(f"\nPlayers with at least 15 output frames: {len(players_with_15plus_frames)}")

# Filter df_output to keep only these players
df_output = df_output.set_index(['game_id', 'play_id', 'nfl_id']).loc[players_with_15plus_frames].reset_index()

print(f"After filtering for at least 15 output frames: {df_output.shape}")
print(f"df_output frame_id range after filtering: {df_output['frame_id'].min()} to {df_output['frame_id'].max()}")

# Also filter df_input_TRUE to keep only players that have at least 15 output frames
print(f"\n--- Filtering df_input_TRUE to match ---")
print(f"Before: {df_input_TRUE.shape}")

df_input_TRUE = df_input_TRUE.set_index(['game_id', 'play_id', 'nfl_id']).loc[
    df_input_TRUE.set_index(['game_id', 'play_id', 'nfl_id']).index.isin(players_with_15plus_frames)
].reset_index()

print(f"After: {df_input_TRUE.shape}")

# Verify we have the same players in both datasets
players_in_input = set(df_input_TRUE[['game_id', 'play_id', 'nfl_id']].apply(tuple, axis=1))
players_in_output = set(df_output[['game_id', 'play_id', 'nfl_id']].apply(tuple, axis=1))

print(f"\nPlayers in df_input_TRUE: {len(players_in_input)}")
print(f"Players in df_output: {len(players_in_output)}")
print(f"Players in both: {len(players_in_input.intersection(players_in_output))}")
print(f"✓ All players have at least 15 output frames (capped at 15)")


Before filtering - df_input_TRUE shape: (1303440, 22)
Before filtering - df_output shape: (562936, 6)
num_frames_output distribution:
num_frames_output
5       7595
6      45811
7     114964
8     144572
9     142384
10    139061
11    110066
12     93127
13     80221
14     65116
15     53207
16     46442
17     36557
18     33164
19     24743
20     23865
21     18926
22     18765
23     19289
24     15470
25     12030
26     11314
27     11889
28     10282
29     10342
30      3733
31      5091
32      1902
33      1414
34       867
36       444
40       315
55       264
94       208
Name: count, dtype: int64


C:\Users\Matteo\AppData\Local\Temp\ipykernel_316\3352722919.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  last_8_frames = df_input_TRUE.groupby(['game_id', 'play_id', 'nfl_id']).apply(



Last 8 frames shape: (368360, 4)
Frames per player:
8    46045
Name: count, dtype: int64
After keeping only last 8 input frames per player: (368360, 22)

--- Processing df_output ---
df_output frame_id range: 1 to 94

Output frame counts distribution:
5       254
6      1749
7      4600
8      5774
9      5441
10     5116
11     3915
12     3184
13     2644
14     2098
15    11270
Name: count, dtype: int64

Players with at least 15 output frames: 11270
After filtering for at least 15 output frames: (169050, 6)
df_output frame_id range after filtering: 1 to 15

--- Filtering df_input_TRUE to match ---
Before: (368360, 22)
After: (90160, 22)

Players in df_input_TRUE: 11270
Players in df_output: 11270
Players in both: 11270
✓ All players have at least 15 output frames (capped at 15)


In [6]:
# Create frame numbering where 0 is the last frame, -1 is second last, ..., -7 is eighth last
# Sort by frame_id within each player to ensure proper ordering
df_input_TRUE = df_input_TRUE.sort_values(['game_id', 'play_id', 'nfl_id', 'frame_id'])
df_input_TRUE['frame_position'] = df_input_TRUE.groupby(['game_id', 'play_id', 'nfl_id']).cumcount() - 7

print(f"Frame positions per player:\n{df_input_TRUE['frame_position'].value_counts().sort_index()}")
print(f"\nSample data with frame_position (note: -7=oldest, ..., -1=second newest, 0=newest):")
print(df_input_TRUE[['game_id', 'play_id', 'nfl_id', 'frame_id', 'frame_position', 'x', 'y']].head(24))

# Pivot df_input_TRUE to wide format - one observation per player
# Grouping identifiers (everything except the variables to pivot)
id_vars = ['game_id', 'play_id', 'player_to_predict', 'nfl_id', 
           'play_direction', 'absolute_yardline_number', 
           'player_height', 'player_weight', 'player_birth_date', 
           'player_position', 'player_side', 'player_role', 
           'num_frames_output', 'ball_land_x', 'ball_land_y']

# Create the pivot using frame_position instead of frame_id
df_wide = df_input_TRUE.pivot_table(
    index=id_vars,
    columns='frame_position',
    values=['x', 'y', 's', 'a', 'dir', 'o'],
    aggfunc='first'  # Use first in case of duplicates
).reset_index()

# Flatten the multi-level column names
# This creates column names like: x_-7, x_-6, ..., x_-1, x_0, y_-7, y_-6, ..., y_-1, y_0, etc.
# Where 0 = last frame, -1 = second last, ..., -7 = eighth last
df_wide.columns = ['_'.join([str(c) for c in col]).strip('_') if col[1] != '' else col[0] 
                   for col in df_wide.columns.values]

print(f"\nOriginal shape: {df_input_TRUE.shape}")
print(f"Wide format shape: {df_wide.shape}")
print(f"\nAll columns: {df_wide.columns.tolist()}")
df_wide.head()


Frame positions per player:
frame_position
-7    11270
-6    11270
-5    11270
-4    11270
-3    11270
-2    11270
-1    11270
 0    11270
Name: count, dtype: int64

Sample data with frame_position (note: -7=oldest, ..., -1=second newest, 0=newest):
       game_id  play_id  nfl_id  frame_id  frame_position      x      y
16  2023090700      101   44930        19              -7  47.13  14.20
17  2023090700      101   44930        20              -6  47.85  14.29
18  2023090700      101   44930        21              -5  48.59  14.35
19  2023090700      101   44930        22              -4  49.34  14.37
20  2023090700      101   44930        23              -3  50.10  14.36
21  2023090700      101   44930        24              -2  50.87  14.32
22  2023090700      101   44930        25              -1  51.65  14.25
23  2023090700      101   44930        26               0  52.43  14.14
0   2023090700      101   46137        19              -7  53.58  19.70
1   2023090700      101   4613

,game_id,play_id,player_to_predict,nfl_id,play_direction,absolute_yardline_number,player_height,player_weight,player_birth_date,player_position,...,x_-1,x_0,y_-7,y_-6,y_-5,y_-4,y_-3,y_-2,y_-1,y_0
0,2023090700,101,True,44930,right,42,6-3,196,1995-02-16,WR,...,51.65,52.43,14.20,14.29,14.35,14.37,14.36,14.32,14.25,14.14
1,2023090700,101,True,46137,right,42,6-1,204,1997-02-15,SS,...,55.45,55.82,19.70,19.49,19.25,18.98,18.69,18.37,18.03,17.67
2,2023090700,101,True,52546,right,42,6-1,193,1997-01-21,CB,...,48.06,48.01,13.19,13.23,13.23,13.19,13.08,12.92,12.70,12.44
3,2023090700,361,True,38696,right,22,6-2,198,1990-03-12,WR,...,34.12,34.19,47.97,47.95,47.95,47.97,48.02,48.09,48.20,48.33
4,2023090700,361,True,46137,right,22,6-1,204,1997-02-15,SS,...,35.61,35.66,46.89,46.78,46.68,46.62,46.60,46.61,46.65,46.72


In [7]:
# Pivot df_output to wide format
print(f"df_output shape before pivot: {df_output.shape}")
print(f"df_output columns: {df_output.columns.tolist()}")
print(f"\nSample df_output:")
print(df_output.head(10))

# Pivot output data - x and y for each future frame
df_output_wide = df_output.pivot_table(
    index=['game_id', 'play_id', 'nfl_id'],
    columns='frame_id',
    values=['x', 'y'],
    aggfunc='first'
).reset_index()

# Flatten column names - this creates x_1, x_2, ..., y_1, y_2, ...
df_output_wide.columns = ['_'.join([str(c) for c in col]).strip('_') if col[1] != '' else col[0] 
                          for col in df_output_wide.columns.values]

print(f"\ndf_output_wide shape: {df_output_wide.shape}")
print(f"Output columns: {[col for col in df_output_wide.columns if col.startswith(('x_', 'y_'))]}")

# Merge with df_wide
df_combined = df_wide.merge(
    df_output_wide,
    on=['game_id', 'play_id', 'nfl_id'],
    how='inner'
)

print(f"\nCombined shape: {df_combined.shape}")
print(f"\nAll columns: {df_combined.columns.tolist()}")

# Reorder columns to have input features first, then outputs
# Get base columns (non-pivoted)
base_cols = ['game_id', 'play_id', 'player_to_predict', 'nfl_id', 
             'play_direction', 'absolute_yardline_number', 
             'player_height', 'player_weight', 'player_birth_date', 
             'player_position', 'player_side', 'player_role', 
             'num_frames_output', 'ball_land_x', 'ball_land_y']

# Get input frame columns (x_-7, x_-6, ..., x_-1, x_0, y_-7, y_-6, ..., y_-1, y_0, etc.)
input_cols = [col for col in df_combined.columns 
              if any(col.endswith(f'_{i}') for i in [-7, -6, -5, -4, -3, -2, -1, 0])]

# Get output frame columns (x_1, x_2, ..., y_1, y_2, ...)
output_cols = [col for col in df_combined.columns 
               if col.startswith(('x_', 'y_')) and col not in input_cols]

# Sort to have proper order: x cols for inputs, then x cols for outputs, then y cols
x_input = sorted([col for col in input_cols if col.startswith('x_')], 
                 key=lambda x: int(x.split('_')[1]))
y_input = sorted([col for col in input_cols if col.startswith('y_')], 
                 key=lambda x: int(x.split('_')[1]))
other_input = [col for col in input_cols if not col.startswith(('x_', 'y_'))]

x_output = sorted([col for col in output_cols if col.startswith('x_')], 
                  key=lambda x: int(x.split('_')[1]))
y_output = sorted([col for col in output_cols if col.startswith('y_')], 
                  key=lambda x: int(x.split('_')[1]))

# Reorder: base + other_input + x_input + x_output + y_input + y_output
df_combined = df_combined[base_cols + other_input + x_input + x_output + y_input + y_output]

print(f"\nReordered columns: {df_combined.columns.tolist()}")
print(f"\nFinal combined dataframe:")
df_combined.head()


df_output shape before pivot: (169050, 6)
df_output columns: ['game_id', 'play_id', 'nfl_id', 'frame_id', 'x', 'y']

Sample df_output:
      game_id  play_id  nfl_id  frame_id      x      y
0  2023090700      101   44930         1  53.20  13.98
1  2023090700      101   44930         2  53.96  13.78
2  2023090700      101   44930         3  54.70  13.54
3  2023090700      101   44930         4  55.41  13.27
4  2023090700      101   44930         5  56.09  12.95
5  2023090700      101   44930         6  56.73  12.58
6  2023090700      101   44930         7  57.35  12.14
7  2023090700      101   44930         8  57.92  11.68
8  2023090700      101   44930         9  58.45  11.17
9  2023090700      101   44930        10  58.95  10.62

df_output_wide shape: (11270, 33)
Output columns: ['x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6', 'x_7', 'x_8', 'x_9', 'x_10', 'x_11', 'x_12', 'x_13', 'x_14', 'x_15', 'y_1', 'y_2', 'y_3', 'y_4', 'y_5', 'y_6', 'y_7', 'y_8', 'y_9', 'y_10', 'y_11', 'y_12', 'y_13', 'y

,game_id,play_id,player_to_predict,nfl_id,play_direction,absolute_yardline_number,player_height,player_weight,player_birth_date,player_position,...,y_6,y_7,y_8,y_9,y_10,y_11,y_12,y_13,y_14,y_15
0,2023090700,101,True,44930,right,42,6-3,196,1995-02-16,WR,...,12.58,12.14,11.68,11.17,10.62,10.02,9.37,8.69,7.99,7.25
1,2023090700,101,True,46137,right,42,6-1,204,1997-02-15,SS,...,15.10,14.57,14.01,13.41,12.80,12.15,11.46,10.75,10.02,9.26
2,2023090700,101,True,52546,right,42,6-1,193,1997-01-21,CB,...,10.00,9.55,9.12,8.70,8.29,8.00,7.70,7.39,7.11,6.86
3,2023090700,361,True,38696,right,22,6-2,198,1990-03-12,WR,...,49.44,49.66,49.88,50.11,50.31,50.48,50.64,50.76,50.84,50.90
4,2023090700,361,True,46137,right,22,6-1,204,1997-02-15,SS,...,47.63,47.84,48.05,48.29,48.55,48.81,49.07,49.33,49.61,49.83


In [8]:
# Standardize movement direction - make all plays go in the same direction (right)
print(f"Play direction distribution before standardization:")
print(df_combined['play_direction'].value_counts())

# Identify which columns need to be flipped for "left" plays
x_cols = [col for col in df_combined.columns if col.startswith('x_')]
y_cols = [col for col in df_combined.columns if col.startswith('y_')]
dir_cols = [col for col in df_combined.columns if col.startswith('dir_')]
o_cols = [col for col in df_combined.columns if col.startswith('o_')]

print(f"\nColumns to standardize:")
print(f"x columns: {x_cols}")
print(f"y columns: {y_cols}")
print(f"dir columns: {dir_cols}")
print(f"o columns: {o_cols}")

# Create a mask for left plays
left_mask = df_combined['play_direction'] == 'left'
print(f"\nNumber of 'left' plays: {left_mask.sum()}")
print(f"Number of 'right' plays: {(~left_mask).sum()}")

# For left plays, flip the coordinates:
# - x: flip horizontally (120 - x, assuming field is 120 yards)
# - y: flip vertically (53.3 - y, assuming field is 53.3 yards wide)
# - dir: add 180 degrees and modulo 360
# - o: add 180 degrees and modulo 360
# - s, a: keep unchanged (magnitudes don't change direction)

# Check sample values before transformation
print(f"\nSample values BEFORE standardization (left play):")
sample_left = df_combined[left_mask].iloc[0]
print(f"play_direction: {sample_left['play_direction']}")
print(f"x_0: {sample_left['x_0']}, y_0: {sample_left['y_0']}")
if 'x_1' in df_combined.columns:
    print(f"x_1: {sample_left['x_1']}, y_1: {sample_left['y_1']}")
print(f"dir_0: {sample_left['dir_0']}, o_0: {sample_left['o_0']}")

# Flip x coordinates for left plays (120 - x)
for col in x_cols:
    df_combined.loc[left_mask, col] = 120 - df_combined.loc[left_mask, col]

# Flip y coordinates for left plays (53.3 - y)
for col in y_cols:
    df_combined.loc[left_mask, col] = 53.3 - df_combined.loc[left_mask, col]

# Flip direction angles for left plays (add 180 and modulo 360)
for col in dir_cols:
    df_combined.loc[left_mask, col] = (df_combined.loc[left_mask, col] + 180) % 360

# Flip orientation angles for left plays (add 180 and modulo 360)
for col in o_cols:
    df_combined.loc[left_mask, col] = (df_combined.loc[left_mask, col] + 180) % 360

# Also flip ball_land_x and ball_land_y
df_combined.loc[left_mask, 'ball_land_x'] = 120 - df_combined.loc[left_mask, 'ball_land_x']
df_combined.loc[left_mask, 'ball_land_y'] = 53.3 - df_combined.loc[left_mask, 'ball_land_y']

# Also flip absolute_yardline_number
df_combined.loc[left_mask, 'absolute_yardline_number'] = 120 - df_combined.loc[left_mask, 'absolute_yardline_number']

print(f"\nSample values AFTER standardization (originally left play):")
sample_left_after = df_combined[left_mask].iloc[0]
print(f"play_direction (original): {sample_left['play_direction']}")
print(f"x_0: {sample_left_after['x_0']}, y_0: {sample_left_after['y_0']}")
if 'x_1' in df_combined.columns:
    print(f"x_1: {sample_left_after['x_1']}, y_1: {sample_left_after['y_1']}")
print(f"dir_0: {sample_left_after['dir_0']}, o_0: {sample_left_after['o_0']}")

print(f"\nStandardization complete! All plays now oriented in the same direction.")
print(f"Shape: {df_combined.shape}")
df_combined.head()


Play direction distribution before standardization:
play_direction
left     5698
right    5572
Name: count, dtype: int64

Columns to standardize:
x columns: ['x_-7', 'x_-6', 'x_-5', 'x_-4', 'x_-3', 'x_-2', 'x_-1', 'x_0', 'x_1', 'x_2', 'x_3', 'x_4', 'x_5', 'x_6', 'x_7', 'x_8', 'x_9', 'x_10', 'x_11', 'x_12', 'x_13', 'x_14', 'x_15']
y columns: ['y_-7', 'y_-6', 'y_-5', 'y_-4', 'y_-3', 'y_-2', 'y_-1', 'y_0', 'y_1', 'y_2', 'y_3', 'y_4', 'y_5', 'y_6', 'y_7', 'y_8', 'y_9', 'y_10', 'y_11', 'y_12', 'y_13', 'y_14', 'y_15']
dir columns: ['dir_-7', 'dir_-6', 'dir_-5', 'dir_-4', 'dir_-3', 'dir_-2', 'dir_-1', 'dir_0']
o columns: ['o_-7', 'o_-6', 'o_-5', 'o_-4', 'o_-3', 'o_-2', 'o_-1', 'o_0']

Number of 'left' plays: 5698
Number of 'right' plays: 5572

Sample values BEFORE standardization (left play):
play_direction: left
x_0: 76.47, y_0: 39.38
x_1: 76.66, y_1: 39.82
dir_0: 25.04, o_0: 75.73

Sample values AFTER standardization (originally left play):
play_direction (original): left
x_0: 43.53, y_0: 1

,game_id,play_id,player_to_predict,nfl_id,play_direction,absolute_yardline_number,player_height,player_weight,player_birth_date,player_position,...,y_6,y_7,y_8,y_9,y_10,y_11,y_12,y_13,y_14,y_15
0,2023090700,101,True,44930,right,42,6-3,196,1995-02-16,WR,...,12.58,12.14,11.68,11.17,10.62,10.02,9.37,8.69,7.99,7.25
1,2023090700,101,True,46137,right,42,6-1,204,1997-02-15,SS,...,15.10,14.57,14.01,13.41,12.80,12.15,11.46,10.75,10.02,9.26
2,2023090700,101,True,52546,right,42,6-1,193,1997-01-21,CB,...,10.00,9.55,9.12,8.70,8.29,8.00,7.70,7.39,7.11,6.86
3,2023090700,361,True,38696,right,22,6-2,198,1990-03-12,WR,...,49.44,49.66,49.88,50.11,50.31,50.48,50.64,50.76,50.84,50.90
4,2023090700,361,True,46137,right,22,6-1,204,1997-02-15,SS,...,47.63,47.84,48.05,48.29,48.55,48.81,49.07,49.33,49.61,49.83


In [9]:
# Convert player height from feet-inches to centimeters
print("Converting player height to centimeters...")
print(f"Sample heights before conversion: {df_combined['player_height'].head(10).tolist()}")

def height_to_cm(height_str):
    """Convert height from 'feet-inches' format to centimeters"""
    if pd.isna(height_str):
        return np.nan
    
    try:
        feet, inches = height_str.split('-')
        feet = int(feet)
        inches = int(inches)
        
        # Convert to centimeters: 1 foot = 30.48 cm, 1 inch = 2.54 cm
        cm = (feet * 30.48) + (inches * 2.54)
        return round(cm, 2)
    except:
        return np.nan

df_combined['player_height_cm'] = df_combined['player_height'].apply(height_to_cm)
print(f"\nSample heights after conversion (cm): {df_combined['player_height_cm'].head(10).tolist()}")

# Drop the original height column
df_combined = df_combined.drop(columns=['player_height'])

# Convert player birth date to age in years
print("\n\nConverting birth date to age...")
print(f"Sample birth dates: {df_combined['player_birth_date'].head(10).tolist()}")

# Extract game date from game_id (format: YYYYMMDD##)
# game_id like 2023090700 -> year=2023, month=09, day=07
def extract_game_date(game_id):
    """Extract date from game_id"""
    game_id_str = str(game_id)
    year = int(game_id_str[0:4])
    month = int(game_id_str[4:6])
    day = int(game_id_str[6:8])
    return pd.Timestamp(year=year, month=month, day=day)

# Get game dates
df_combined['game_date'] = df_combined['game_id'].apply(extract_game_date)

# Convert birth_date to datetime
df_combined['player_birth_date'] = pd.to_datetime(df_combined['player_birth_date'])

# Calculate age at time of game
df_combined['player_age'] = (df_combined['game_date'] - df_combined['player_birth_date']).dt.days / 365.25

# Round to 2 decimal places
df_combined['player_age'] = df_combined['player_age'].round(2)

print(f"\nSample ages: {df_combined['player_age'].head(10).tolist()}")
print(f"Age statistics:\n{df_combined['player_age'].describe()}")

# Drop temporary columns
df_combined = df_combined.drop(columns=['player_birth_date', 'game_date'])

print(f"\nConversions complete!")
print(f"Shape: {df_combined.shape}")
print(f"\nUpdated columns (first 25): {df_combined.columns[:25].tolist()}")
df_combined.head()


Converting player height to centimeters...
Sample heights before conversion: ['6-3', '6-1', '6-1', '6-2', '6-1', '5-10', '5-11', '6-3', '5-11', '6-1']

Sample heights after conversion (cm): [190.5, 185.42, 185.42, 187.96, 185.42, 177.8, 180.34, 190.5, 180.34, 185.42]


Converting birth date to age...
Sample birth dates: ['1995-02-16', '1997-02-15', '1997-01-21', '1990-03-12', '1997-02-15', '1996-05-18', '1995-02-27', '1996-04-04', '1997-12-20', '2000-11-14']

Sample ages: [28.56, 26.56, 26.63, 33.49, 26.56, 27.3, 28.53, 27.43, 25.71, 22.81]
Age statistics:
count    11270.000000
mean        26.487487
std          2.822782
min         21.200000
25%         24.320000
50%         26.090000
75%         28.320000
max         35.610000
Name: player_age, dtype: float64

Conversions complete!
Shape: (11270, 93)

Updated columns (first 25): ['game_id', 'play_id', 'player_to_predict', 'nfl_id', 'play_direction', 'absolute_yardline_number', 'player_weight', 'player_position', 'player_side', 'playe

,game_id,play_id,player_to_predict,nfl_id,play_direction,absolute_yardline_number,player_weight,player_position,player_side,player_role,...,y_8,y_9,y_10,y_11,y_12,y_13,y_14,y_15,player_height_cm,player_age
0,2023090700,101,True,44930,right,42,196,WR,Offense,Targeted Receiver,...,11.68,11.17,10.62,10.02,9.37,8.69,7.99,7.25,190.50,28.56
1,2023090700,101,True,46137,right,42,204,SS,Defense,Defensive Coverage,...,14.01,13.41,12.80,12.15,11.46,10.75,10.02,9.26,185.42,26.56
2,2023090700,101,True,52546,right,42,193,CB,Defense,Defensive Coverage,...,9.12,8.70,8.29,8.00,7.70,7.39,7.11,6.86,185.42,26.63
3,2023090700,361,True,38696,right,22,198,WR,Offense,Targeted Receiver,...,49.88,50.11,50.31,50.48,50.64,50.76,50.84,50.90,187.96,33.49
4,2023090700,361,True,46137,right,22,204,SS,Defense,Defensive Coverage,...,48.05,48.29,48.55,48.81,49.07,49.33,49.61,49.83,185.42,26.56


In [10]:
"""
================================================================================
TRAIN/TEST SPLIT STRATEGY
================================================================================

CRITICAL: We split by game_id to prevent data leakage
- All players from a game stay together (either train or test)
- This ensures no information about a test game appears in training

SPLIT RATIO: 80/20 (4/5 for training, 1/5 for testing)

NEXT STEPS:
- df_combined will be split into train and test portions
- Train portion → df_ready (with column cleaning)
- Test portion → detailed split for proper evaluation (next cell)
================================================================================
"""

# Split data: 1/5 for test, 4/5 for training (based on game_id)
print(f"Original df_combined shape: {df_combined.shape}")
print(f"Unique game_ids: {df_combined['game_id'].nunique()}")

# Get unique game_ids
unique_games = df_combined['game_id'].unique()
print(f"\nTotal unique games: {len(unique_games)}")

# Sample 1/5 of game_ids for test set
np.random.seed(42)  # For reproducibility
n_test_games = int(len(unique_games) * 0.2)  # 1/5 = 20%
test_games = np.random.choice(unique_games, size=n_test_games, replace=False)

print(f"Number of games in test set: {len(test_games)}")
print(f"Number of games in training set: {len(unique_games) - len(test_games)}")

# Split the data - keep test data with ALL information for now
df_test_full = df_combined[df_combined['game_id'].isin(test_games)].copy()
df_train_full = df_combined[~df_combined['game_id'].isin(test_games)].copy()

# Verify no overlap BEFORE dropping columns
assert len(set(df_test_full['game_id']).intersection(set(df_train_full['game_id']))) == 0, "Error: Overlap between test and training game_ids!"
print("\n✓ Verified: No overlap between test and training game_ids")
print(f"Test set games: {sorted(df_test_full['game_id'].unique())}")

# For training: remove identifier columns and unnecessary metadata to create df_ready
# CRITICAL: game_id, play_id, nfl_id are identifiers, NOT features!
# Training a model with these would cause overfitting to specific games/plays/players
columns_to_remove = ['game_id', 'play_id', 'nfl_id', 'player_to_predict', 
                     'play_direction', 'num_frames_output', 'ball_land_x', 'ball_land_y']
df_ready = df_train_full.drop(columns=columns_to_remove)

print(f"\nAfter split and column removal:")
print(f"df_test_full shape: {df_test_full.shape} (keeps ALL columns for proper evaluation)")
print(f"df_ready shape: {df_ready.shape} (training data, removed {len(columns_to_remove)} columns)")

# Encode categorical variables for LightGBM
print(f"\n--- Encoding categorical variables ---")
from sklearn.preprocessing import LabelEncoder

categorical_cols = ['player_position', 'player_side', 'player_role']

# Check what we have
print(f"Checking categorical columns in df_ready:")
for col in categorical_cols:
    if col in df_ready.columns:
        print(f"  {col}: dtype={df_ready[col].dtype}, unique values={df_ready[col].nunique()}")

# Store encoders for later use on test data
encoders = {}

for col in categorical_cols:
    if col in df_ready.columns:
        le = LabelEncoder()
        df_ready[col] = le.fit_transform(df_ready[col].astype(str))
        encoders[col] = le
        print(f"  ✓ Encoded {col}: {len(le.classes_)} unique categories")

print(f"\ndf_ready after encoding:")
print(f"  Shape: {df_ready.shape}")
print(f"  Data types:")
print(df_ready.dtypes)


Original df_combined shape: (11270, 93)
Unique game_ids: 272

Total unique games: 272
Number of games in test set: 54
Number of games in training set: 218

✓ Verified: No overlap between test and training game_ids
Test set games: [np.int64(2023091005), np.int64(2023091008), np.int64(2023091100), np.int64(2023091702), np.int64(2023091705), np.int64(2023091707), np.int64(2023091708), np.int64(2023091800), np.int64(2023092409), np.int64(2023092412), np.int64(2023092500), np.int64(2023100111), np.int64(2023100802), np.int64(2023100803), np.int64(2023100900), np.int64(2023101500), np.int64(2023101503), np.int64(2023101511), np.int64(2023101900), np.int64(2023102210), np.int64(2023102902), np.int64(2023102906), np.int64(2023102909), np.int64(2023102911), np.int64(2023102912), np.int64(2023102913), np.int64(2023110502), np.int64(2023110504), np.int64(2023111200), np.int64(2023111207), np.int64(2023111209), np.int64(2023111901), np.int64(2023111907), np.int64(2023112301), np.int64(2023113000),

In [11]:
"""
================================================================================
PREPARE TRAINING DATA FOR KNeighborsRegressor
================================================================================

Since all players have exactly 15 output frames, we create:
- X_train: Input features only
- y_train: All 30 output values (x_1 to x_15, y_1 to y_15)

================================================================================
"""

print("="*80)
print("PREPARING TRAINING DATA FOR KNeighborsRegressor")
print("="*80)

print(f"\ndf_ready shape: {df_ready.shape}")

# Separate input features from output values
x_output_cols = [f'x_{i}' for i in range(1, 16)]
y_output_cols = [f'y_{i}' for i in range(1, 16)]
all_output_cols = x_output_cols + y_output_cols

# X_train: All columns except the 30 output columns
X_train = df_ready.drop(columns=all_output_cols)

# y_train: Only the 30 output columns in specific order
y_train = df_ready[all_output_cols].copy()

print(f"\n✓ Training data prepared:")
print(f"   X_train shape: {X_train.shape}")
print(f"   y_train shape: {y_train.shape}")
print(f"   y_train columns (first 5, last 5): {y_train.columns[:5].tolist()} ... {y_train.columns[-5:].tolist()}")

# Verify no NAs in y_train
na_count = y_train.isna().sum().sum()
print(f"\n   Total NAs in y_train: {na_count} (should be 0)")

if na_count == 0:
    print(f"   ✓ No NAs found - ready for KNeighborsRegressor!")
else:
    print(f"   ⚠️  WARNING: Found {na_count} NAs in y_train!")

print("\n" + "="*80)



PREPARING TRAINING DATA FOR KNeighborsRegressor

df_ready shape: (8938, 85)

✓ Training data prepared:
   X_train shape: (8938, 55)
   y_train shape: (8938, 30)
   y_train columns (first 5, last 5): ['x_1', 'x_2', 'x_3', 'x_4', 'x_5'] ... ['y_11', 'y_12', 'y_13', 'y_14', 'y_15']

   Total NAs in y_train: 0 (should be 0)
   ✓ No NAs found - ready for KNeighborsRegressor!



In [12]:
"""
================================================================================
PREPARE TEST DATA FOR KNeighborsRegressor
================================================================================

Prepare test data in the same format as training data:
- X_test: Input features only
- y_test: All 30 output values

================================================================================
"""

print("="*80)
print("PREPARING TEST DATA FOR KNeighborsRegressor")
print("="*80)

print(f"\ndf_test_full shape: {df_test_full.shape}")

# Separate input features from output values
x_output_cols = [f'x_{i}' for i in range(1, 16)]
y_output_cols = [f'y_{i}' for i in range(1, 16)]
all_output_cols = x_output_cols + y_output_cols

# Remove metadata columns to create X_test
cols_to_remove_for_X = all_output_cols + ['game_id', 'play_id', 'nfl_id', 'player_to_predict', 
                                            'play_direction', 'num_frames_output', 'ball_land_x', 'ball_land_y']
cols_to_remove_for_X = [col for col in cols_to_remove_for_X if col in df_test_full.columns]

X_test = df_test_full.drop(columns=cols_to_remove_for_X)

# Encode categorical variables using the same encoders from training
print(f"\nEncoding categorical variables in test data...")
for col in categorical_cols:
    if col in X_test.columns and col in encoders:
        # Handle unseen categories by assigning them to -1
        X_test[col] = X_test[col].astype(str)
        X_test[col] = X_test[col].apply(
            lambda x: encoders[col].transform([x])[0] if x in encoders[col].classes_ else -1
        )
        print(f"   ✓ Encoded {col}")

# y_test: Only the 30 output columns
y_test = df_test_full[all_output_cols].copy()

print(f"\n✓ Test data prepared:")
print(f"   X_test shape: {X_test.shape}")
print(f"   y_test shape: {y_test.shape}")

# Verify no NAs in y_test
na_count = y_test.isna().sum().sum()
print(f"\n   Total NAs in y_test: {na_count} (should be 0)")

# Verify column alignment between train and test
assert list(X_train.columns) == list(X_test.columns), "Column mismatch between X_train and X_test!"
assert list(y_train.columns) == list(y_test.columns), "Column mismatch between y_train and y_test!"
print(f"\n✓ Verified column alignment between train and test")

print(f"\n{'='*80}")
print(f"DATA PREPARATION COMPLETE")
print(f"{'='*80}")
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")
print(f"\n✓ Ready for KNeighborsRegressor!")



PREPARING TEST DATA FOR KNeighborsRegressor

df_test_full shape: (2332, 93)

Encoding categorical variables in test data...
   ✓ Encoded player_position
   ✓ Encoded player_side
   ✓ Encoded player_role

✓ Test data prepared:
   X_test shape: (2332, 55)
   y_test shape: (2332, 30)

   Total NAs in y_test: 0 (should be 0)

✓ Verified column alignment between train and test

DATA PREPARATION COMPLETE
X_train: (8938, 55), y_train: (8938, 30)
X_test: (2332, 55), y_test: (2332, 30)

✓ Ready for KNeighborsRegressor!


In [50]:
"""
================================================================================
TRAIN KNeighborsRegressor MODEL
================================================================================

Train a single KNeighborsRegressor that predicts all 30 output values at once.

Parameters:
- n_neighbors: Number of neighbors (default=5)
- weights: 'distance' weights closer neighbors more heavily
- metric: 'euclidean' for standard distance
- n_jobs: -1 to use all CPU cores

================================================================================
"""

from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import time

print("="*80)
print("TRAINING KNeighborsRegressor")
print("="*80)

# Create the model
knn = KNeighborsRegressor(
    n_neighbors=5,
    weights='distance',  # Weight by inverse distance
    metric='euclidean',
    n_jobs=-1  # Use all available cores
)

print(f"\nModel configuration:")
print(f"  Algorithm: KNeighborsRegressor")
print(f"  n_neighbors: {knn.n_neighbors}")
print(f"  weights: {knn.weights}")
print(f"  metric: {knn.metric}")

# Train the model
print(f"\nTraining on {X_train.shape[0]} samples...")
print(f"  Input features: {X_train.shape[1]}")
print(f"  Output values: {y_train.shape[1]}")

start_time = time.time()
knn.fit(X_train, y_train)
training_time = time.time() - start_time

print(f"\n✓ Training complete in {training_time:.2f} seconds")

print("\n" + "="*80)
print("MODEL TRAINING COMPLETE")
print("="*80)



TRAINING KNeighborsRegressor

Model configuration:
  Algorithm: KNeighborsRegressor
  n_neighbors: 5
  weights: distance
  metric: euclidean

Training on 8938 samples...
  Input features: 55
  Output values: 30

✓ Training complete in 0.01 seconds

MODEL TRAINING COMPLETE


In [51]:

"""
================================================================================
EVALUATE ON TRAINING DATA
================================================================================

Check model performance on training set to understand fitting quality.

================================================================================
"""

print("="*80)
print("EVALUATING ON TRAINING DATA")
print("="*80)

# Make predictions on training data
print(f"\nMaking predictions on {X_train.shape[0]} training samples...")
start_time = time.time()
y_train_pred = knn.predict(X_train)
pred_time = time.time() - start_time

print(f"✓ Predictions complete in {pred_time:.2f} seconds")
print(f"  Average time per sample: {pred_time/X_train.shape[0]*1000:.2f} ms")

# Calculate overall metrics
train_mae = mean_absolute_error(y_train, y_train_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))

print(f"\n{'='*80}")
print(f"OVERALL TRAINING METRICS")
print(f"{'='*80}")
print(f"  Overall MAE:  {train_mae:.4f} yards")
print(f"  Overall RMSE: {train_rmse:.4f} yards")

# Calculate per-frame metrics
print(f"\n{'='*80}")
print(f"PER-FRAME METRICS (Training)")
print(f"{'='*80}")

x_cols_names = [f'x_{i}' for i in range(1, 16)]
y_cols_names = [f'y_{i}' for i in range(1, 16)]

print(f"\nX-coordinate predictions (frames 1-15):")
x_train_true = y_train[x_cols_names].values
x_train_pred = y_train_pred[:, :15]
for i in range(15):
    mae = mean_absolute_error(x_train_true[:, i], x_train_pred[:, i])
    print(f"  Frame {i+1:2d}: MAE = {mae:.4f} yards")

print(f"\nY-coordinate predictions (frames 1-15):")
y_train_true = y_train[y_cols_names].values
y_train_pred_y = y_train_pred[:, 15:]
for i in range(15):
    mae = mean_absolute_error(y_train_true[:, i], y_train_pred_y[:, i])
    print(f"  Frame {i+1:2d}: MAE = {mae:.4f} yards")

# Calculate average MAE and RMSE for x and y separately
avg_x_mae_train = mean_absolute_error(x_train_true.flatten(), x_train_pred.flatten())
avg_y_mae_train = mean_absolute_error(y_train_true.flatten(), y_train_pred_y.flatten())
avg_x_rmse_train = np.sqrt(mean_squared_error(x_train_true.flatten(), x_train_pred.flatten()))
avg_y_rmse_train = np.sqrt(mean_squared_error(y_train_true.flatten(), y_train_pred_y.flatten()))

print(f"\n{'='*80}")
print(f"SUMMARY - Training Performance")
print(f"{'='*80}")
print(f"  X-coordinates - Average MAE:  {avg_x_mae_train:.4f} yards")
print(f"  X-coordinates - Average RMSE: {avg_x_rmse_train:.4f} yards")
print(f"  Y-coordinates - Average MAE:  {avg_y_mae_train:.4f} yards")
print(f"  Y-coordinates - Average RMSE: {avg_y_rmse_train:.4f} yards")
print(f"  Overall MAE:  {train_mae:.4f} yards")
print(f"  Overall RMSE: {train_rmse:.4f} yards")

EVALUATING ON TRAINING DATA

Making predictions on 8938 training samples...
✓ Predictions complete in 0.23 seconds
  Average time per sample: 0.03 ms

OVERALL TRAINING METRICS
  Overall MAE:  0.0000 yards
  Overall RMSE: 0.0000 yards

PER-FRAME METRICS (Training)

X-coordinate predictions (frames 1-15):
  Frame  1: MAE = 0.0000 yards
  Frame  2: MAE = 0.0000 yards
  Frame  3: MAE = 0.0000 yards
  Frame  4: MAE = 0.0000 yards
  Frame  5: MAE = 0.0000 yards
  Frame  6: MAE = 0.0000 yards
  Frame  7: MAE = 0.0000 yards
  Frame  8: MAE = 0.0000 yards
  Frame  9: MAE = 0.0000 yards
  Frame 10: MAE = 0.0000 yards
  Frame 11: MAE = 0.0000 yards
  Frame 12: MAE = 0.0000 yards
  Frame 13: MAE = 0.0000 yards
  Frame 14: MAE = 0.0000 yards
  Frame 15: MAE = 0.0000 yards

Y-coordinate predictions (frames 1-15):
  Frame  1: MAE = 0.0000 yards
  Frame  2: MAE = 0.0000 yards
  Frame  3: MAE = 0.0000 yards
  Frame  4: MAE = 0.0000 yards
  Frame  5: MAE = 0.0000 yards
  Frame  6: MAE = 0.0000 yards
  F

In [52]:
"""
================================================================================
EVALUATE ON TEST DATA
================================================================================

Evaluate model performance on unseen test data.

================================================================================
"""

print("="*80)
print("EVALUATING ON TEST DATA")
print("="*80)

# Make predictions on test data
print(f"\nMaking predictions on {X_test.shape[0]} test samples...")
start_time = time.time()
y_test_pred = knn.predict(X_test)
test_pred_time = time.time() - start_time

print(f"✓ Predictions complete in {test_pred_time:.2f} seconds")
print(f"  Average time per sample: {test_pred_time/X_test.shape[0]*1000:.2f} ms")

# Calculate overall metrics
test_mae = mean_absolute_error(y_test, y_test_pred)
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

print(f"\n{'='*80}")
print(f"OVERALL TEST METRICS")
print(f"{'='*80}")
print(f"  Overall MAE:  {test_mae:.4f} yards")
print(f"  Overall RMSE: {test_rmse:.4f} yards")

# Calculate per-frame metrics
print(f"\n{'='*80}")
print(f"PER-FRAME METRICS (Test)")
print(f"{'='*80}")

x_cols_names = [f'x_{i}' for i in range(1, 16)]
y_cols_names = [f'y_{i}' for i in range(1, 16)]

print(f"\nX-coordinate predictions (frames 1-15):")
x_test_true = y_test[x_cols_names].values
x_test_pred = y_test_pred[:, :15]
x_test_mae_per_frame = []
for i in range(15):
    mae = mean_absolute_error(x_test_true[:, i], x_test_pred[:, i])
    x_test_mae_per_frame.append(mae)
    print(f"  Frame {i+1:2d}: MAE = {mae:.4f} yards")

print(f"\nY-coordinate predictions (frames 1-15):")
y_test_true = y_test[y_cols_names].values
y_test_pred_y = y_test_pred[:, 15:]
y_test_mae_per_frame = []
for i in range(15):
    mae = mean_absolute_error(y_test_true[:, i], y_test_pred_y[:, i])
    y_test_mae_per_frame.append(mae)
    print(f"  Frame {i+1:2d}: MAE = {mae:.4f} yards")

# Calculate average MAE and RMSE for x and y separately
avg_x_mae_test = mean_absolute_error(x_test_true.flatten(), x_test_pred.flatten())
avg_y_mae_test = mean_absolute_error(y_test_true.flatten(), y_test_pred_y.flatten())
avg_x_rmse_test = np.sqrt(mean_squared_error(x_test_true.flatten(), x_test_pred.flatten()))
avg_y_rmse_test = np.sqrt(mean_squared_error(y_test_true.flatten(), y_test_pred_y.flatten()))

print(f"\n{'='*80}")
print(f"SUMMARY - Test Performance")
print(f"{'='*80}")
print(f"  X-coordinates - Average MAE:  {avg_x_mae_test:.4f} yards")
print(f"  X-coordinates - Average RMSE: {avg_x_rmse_test:.4f} yards")
print(f"  Y-coordinates - Average MAE:  {avg_y_mae_test:.4f} yards")
print(f"  Y-coordinates - Average RMSE: {avg_y_rmse_test:.4f} yards")
print(f"  Overall MAE:  {test_mae:.4f} yards")
print(f"  Overall RMSE: {test_rmse:.4f} yards")

# Compare train vs test
print(f"\n{'='*80}")
print(f"TRAIN vs TEST COMPARISON")
print(f"{'='*80}")
print(f"  Training RMSE:  {train_rmse:.4f} yards")
print(f"  Test RMSE:      {test_rmse:.4f} yards")
print(f"  Difference:     {test_rmse - train_rmse:.4f} yards")
print(f"  Relative diff:  {((test_rmse - train_rmse) / train_rmse * 100):.2f}%")

if test_mae - train_mae < 0.5:
    print(f"\n  ✓ Good generalization - low overfitting")
elif test_mae - train_mae < 1.0:
    print(f"\n  ⚠ Moderate overfitting detected")
else:
    print(f"\n  ⚠⚠ Significant overfitting")

EVALUATING ON TEST DATA

Making predictions on 2332 test samples...
✓ Predictions complete in 0.21 seconds
  Average time per sample: 0.09 ms

OVERALL TEST METRICS
  Overall MAE:  3.9140 yards
  Overall RMSE: 5.6072 yards

PER-FRAME METRICS (Test)

X-coordinate predictions (frames 1-15):
  Frame  1: MAE = 3.7425 yards
  Frame  2: MAE = 3.7557 yards
  Frame  3: MAE = 3.7719 yards
  Frame  4: MAE = 3.7916 yards
  Frame  5: MAE = 3.8171 yards
  Frame  6: MAE = 3.8481 yards
  Frame  7: MAE = 3.8852 yards
  Frame  8: MAE = 3.9308 yards
  Frame  9: MAE = 3.9835 yards
  Frame 10: MAE = 4.0441 yards
  Frame 11: MAE = 4.1148 yards
  Frame 12: MAE = 4.1943 yards
  Frame 13: MAE = 4.2838 yards
  Frame 14: MAE = 4.3840 yards
  Frame 15: MAE = 4.4952 yards

Y-coordinate predictions (frames 1-15):
  Frame  1: MAE = 3.5747 yards
  Frame  2: MAE = 3.5831 yards
  Frame  3: MAE = 3.5947 yards
  Frame  4: MAE = 3.6110 yards
  Frame  5: MAE = 3.6328 yards
  Frame  6: MAE = 3.6605 yards
  Frame  7: MAE = 3